In [2]:
import os

from ipynb.fs.defs.data_process import make_noise_playlist



In [3]:
from ipynb.fs.defs.data_process import load_clips, make_word2ind

clips = load_clips()

word2ind = make_word2ind(clips)
print(f'{len(word2ind)} words from {len(clips)} clips')
word2ind

237 words from 20003 clips


{'largest': 0,
 'same': 1,
 'north': 2,
 'works': 3,
 'william': 4,
 'book': 5,
 'died': 6,
 'fiction': 7,
 'over': 8,
 'england': 9,
 'canada': 10,
 'among': 11,
 'being': 12,
 'language': 13,
 'political': 14,
 'work': 15,
 'life': 16,
 'this': 17,
 'developed': 18,
 'more': 19,
 'system': 20,
 'october': 21,
 'line': 22,
 'such': 23,
 'well': 24,
 'king': 25,
 'have': 26,
 'english': 27,
 'high': 28,
 'video': 29,
 'series': 30,
 'states': 31,
 'made': 32,
 'each': 33,
 'while': 34,
 'british': 35,
 'group': 36,
 'various': 37,
 'club': 38,
 'history': 39,
 'november': 40,
 'school': 41,
 'sometimes': 42,
 'term': 43,
 'because': 44,
 'international': 45,
 'known': 46,
 'american': 47,
 'central': 48,
 'eighty': 49,
 'union': 50,
 'form': 51,
 'house': 52,
 'since': 53,
 'built': 54,
 'early': 55,
 'became': 56,
 'began': 57,
 'long': 58,
 'including': 59,
 'eighteen': 60,
 'found': 61,
 'period': 62,
 'park': 63,
 'million': 64,
 'generally': 65,
 'produced': 66,
 'nine': 67,
 'hun

c:\Users\nwong\Documents\nick-repo\.conda\Lib\site-packages\pycochleagram-0.1-py3.11.egg\pycochleagram\cochleagram.py:135: RuntimeWarning: divide by zero encountered in log10
  freqs_to_plot = np.log10(freqs)


KeyboardInterrupt: 